### Entrenamiento del modelo YOLOv7

In [ ]:
BASE_PATH = "/tf/notebooks/CEIA/computer-vision-2/tp-final/models/yolov7"

import os

# !pip install -r {os.path.join(BASE_PATH, "yolov7/requirements.txt")}
# !apt-get update
# !apt-get install ffmpeg libsm6 libxext6 -y

import torch

Descarga del modelo pre-entrenado

In [ ]:
PRETRAINED_PATH = os.path.join(BASE_PATH, "yolov7-pretrained")
WEIGHTS_FILE = os.path.join(PRETRAINED_PATH, "yolov7_training.pt")

if os.access(WEIGHTS_FILE, os.F_OK) is False:
    try:
        original_umask = os.umask(0)
        os.mkdir(PRETRAINED_PATH, mode=0o775)
    except FileExistsError:
        pass
    finally:
        os.umask(original_umask)

    # download COCO starting checkpoint
    !wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt -P {PRETRAINED_PATH}

Definición de datasets y carga de hiperparámetros

In [ ]:
RUNS_PATH = "/tf/notebooks/CEIA/computer-vision-2/tp-final/runs"
TRAIN_PROJECT_PATH = os.path.join(RUNS_PATH, "train")
TRAIN_SCRIPT = os.path.join(BASE_PATH, "yolov7/train.py")
DATASET_PATH = "/tf/datasets/final-dataset/yolov7-dataset"
DATA_YAML = os.path.join(DATASET_PATH, "data.yaml")

NUM_BATCHES = 2
NUM_EPOCHS = 50
FREEZE_LAYERS = 50 # backbone
IMG_SIZE = 640
experiment = "yolov7-50-epochs-640"

Entrenamiento

In [ ]:
torch.cuda.empty_cache() 

!python {TRAIN_SCRIPT} \
--project {TRAIN_PROJECT_PATH} \
--name {experiment} \
--batch {NUM_BATCHES} \
--epochs {NUM_EPOCHS} \
--data {DATA_YAML} \
--img-size {IMG_SIZE} {IMG_SIZE} \
--weights {WEIGHTS_FILE} \
--device 0 \
--freeze {FREEZE_LAYERS}

In [ ]:
# run tensorboard
# !tensorboard --logdir=/tf/notebooks/CEIA/computer-vision-2/tp-final/runs/train --host 0.0.0.0

Testeo

In [ ]:
TEST_PROJECT_PATH = os.path.join(TRAIN_PROJECT_PATH, f"{experiment}")
TEST_SCRIPT = os.path.join(BASE_PATH, "yolov7/test.py")
TEST_WEIGHTS = os.path.join(TRAIN_PROJECT_PATH, f"{experiment}/weights/best.pt")
TEST_DATA_PATH = os.path.join(DATASET_PATH, "test/images")
TEST_NAME = 'test'

CONFIDENCE_THRESHOLD = 0.7

!python {TEST_SCRIPT} \
 --project {TEST_PROJECT_PATH} \
 --name {TEST_NAME} \
 --batch {NUM_BATCHES} \
 --task test \
 --data {DATA_YAML} \
 --img-size {IMG_SIZE} \
 --weights {TEST_WEIGHTS} \
 --conf-thres {CONFIDENCE_THRESHOLD} \
 --device 0 \
 --verbose

Inferencia

In [ ]:
DETECT_SCRIPT = os.path.join(BASE_PATH, "yolov7/detect.py")
DETECT_DATA_PATH = os.path.join(DATASET_PATH, "test/images")
DETECT_NAME = 'detect'

!python {DETECT_SCRIPT} \
 --project {TEST_PROJECT_PATH} \
 --name {DETECT_NAME} \
 --weights {TEST_WEIGHTS} \
 --conf-thres {CONFIDENCE_THRESHOLD} \
 --source {DETECT_DATA_PATH}